# Weather Data Analysis for Pune

This notebook fetches weather data using the OpenWeatherMap API and performs analysis using pandas and matplotlib.

In [8]:
import requests
import pandas as pd
import psycopg2
from datetime import datetime

try:
    url = f'https://api.openweathermap.org/data/2.5/forecast?q=Pune&appid=4a3d30bd5afd12dfabd8eb4b30be3809&units=metric'
    
    print("Fetching weather data...")
    response = requests.get(url)
    response.raise_for_status()
    weather_json = response.json()
    
    data = []
    for item in weather_json['list']:
        data.append({
            'datetime': datetime.fromtimestamp(item['dt']),
            'temp': item['main']['temp'],
            'humidity': item['main']['humidity']
        })
    
    df = pd.DataFrame(data)
    
    print("\nWeather data fetched successfully. Sample data:")
    print(df.head(2))
    
    print("\nConnecting to PostgreSQL...")
    conn = psycopg2.connect(
        dbname="tejas",
        user="postgres",
        password="tejas2027",
        host="localhost",
        port="5432"
    )
    
    cur = conn.cursor()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS weather_data (
        id SERIAL PRIMARY KEY,
        city VARCHAR(100),
        datetime TIMESTAMP,
        temp FLOAT,
        humidity FLOAT
    );
    """
    cur.execute(create_table_query)
    
    print("\nInserting data into database.")
    for _, row in df.iterrows():
        cur.execute(
            "INSERT INTO weather_data (city, datetime, temp, humidity) VALUES (%s, %s, %s, %s)",
            ("Pune", row['datetime'], row['temp'], row['humidity'])
        )
    
    conn.commit()
    print("Data successfully stored in PostgreSQL database!")

except requests.exceptions.RequestException as e:
    print(f"Error fetching weather data: {e}")
except psycopg2.Error as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()
        print("Database connection closed.")

Fetching weather data...

Weather data fetched successfully. Sample data:
             datetime   temp  humidity
0 2025-09-21 20:30:00  24.18        90
1 2025-09-21 23:30:00  23.32        92

Connecting to PostgreSQL...

Inserting data into database.
Data successfully stored in PostgreSQL database!
Database connection closed.

Weather data fetched successfully. Sample data:
             datetime   temp  humidity
0 2025-09-21 20:30:00  24.18        90
1 2025-09-21 23:30:00  23.32        92

Connecting to PostgreSQL...

Inserting data into database.
Data successfully stored in PostgreSQL database!
Database connection closed.


In [ ]:
import numpy as np 
from sklearn.linear_model import LinearRegression

# Add a "day" column 
df['day'] = np.arange(len(df))

X = df[["day"]]
y = df["temp"]


model = LinearRegression()
model.fit(X, y)

# Predict the next 10 days
future_days = np.arange(len(df), len(df) + 10).reshape(-1, 1)
predictions = model.predict(future_days)

print("The Predicted Temperatures for Next 10 days in Pune are as follows:\n")

for i, temp in enumerate(predictions, 1):
    print(f"Day +{i}: {temp:.2f} °F")


The Predicted Temperatures for Next 10 days in Pune are as follows:

Day +1: 23.66 °F
Day +2: 23.67 °F
Day +3: 23.68 °F
Day +4: 23.68 °F
Day +5: 23.69 °F
Day +6: 23.70 °F
Day +7: 23.70 °F
Day +8: 23.71 °F
Day +9: 23.72 °F
Day +10: 23.72 °F


c:\Users\tejas\OneDrive\Desktop\DE\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
